In [ ]:
!pip install transformers

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import tensorflow_addons as tfa
from collections import Counter
from datetime import datetime
import calendar

In [ ]:
# create bert model

from transformers import BertConfig
from transformers import TFBertForSequenceClassification

model_name = 'bert-base-uncased'
config1 = BertConfig.from_pretrained(model_name)
config1.num_labels=2

bert_model = TFBertForSequenceClassification.from_pretrained(model_name,config = config1)
learning_rate = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
# model.get_layer('bert').trainable=False

In [ ]:
# Train just the bert
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( bert_posts, bert_labels, test_size=0.3, random_state=21)

bert_train_dataset = encode_examples(X_train,y_train)
bert_test_dataset = encode_examples(X_test,y_test)
print(len(bert_train_dataset))
bert_train_dataset_batched = bert_train_dataset.batch(batch_size=5)
bert_test_dataset_batched = bert_test_dataset.batch(batch_size=5)

9462


In [ ]:
bert_history = bert_model.fit(bert_train_dataset_batched,epochs=4,verbose=1,validation_data=bert_test_dataset_batched)

Epoch 1/4
1893/1893 [==============================] - 198s 105ms/step - loss: 0.6499 - accuracy: 0.6260 - val_loss: 0.6227 - val_accuracy: 0.6544
Epoch 2/4
1893/1893 [==============================] - 197s 104ms/step - loss: 0.5716 - accuracy: 0.7151 - val_loss: 0.6478 - val_accuracy: 0.6484
Epoch 3/4
1893/1893 [==============================] - 200s 105ms/step - loss: 0.4350 - accuracy: 0.8117 - val_loss: 0.8929 - val_accuracy: 0.6467
Epoch 4/4
1893/1893 [==============================] - 199s 105ms/step - loss: 0.3170 - accuracy: 0.8746 - val_loss: 1.0662 - val_accuracy: 0.6415


In [ ]:
res = np.argmax(bert_model.predict(bert_test_dataset_batched),axis = -1)
lebl = []
for data in bert_test_dataset_batched:
  lebl.extend(tf.squeeze(data[1]).numpy())

In [ ]:
print(res[0][:100])
lebl = []
for i,data in enumerate(bert_test_dataset_batched):
  lebb = np.squeeze(data[1])
  if i!= len(bert_test_dataset_batched) -1:
    lebl.extend(list(lebb))
print(lebl[:100])

[0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0
 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 1 0 0]
[1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0]


In [ ]:
bert_history = bert_model.fit(bert_train_dataset_batched,epochs=4,verbose=1)

Epoch 1/4
2700/2700 [==============================] - 242s 89ms/step - loss: 0.6726 - accuracy: 0.5799
Epoch 2/4
2700/2700 [==============================] - 247s 91ms/step - loss: 0.6337 - accuracy: 0.6490
Epoch 3/4
2700/2700 [==============================] - 247s 92ms/step - loss: 0.5601 - accuracy: 0.7255
Epoch 4/4
2700/2700 [==============================] - 247s 92ms/step - loss: 0.4393 - accuracy: 0.8079


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_length_text = 50
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  },label

def encode_examples(posts,labels):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
  for post,label in zip(posts,labels):
    if not isinstance(post, str):
      continue
    bert_input = tokenizer.encode_plus(post,
                                      truncation=True,           
                                      add_special_tokens = True, # add [CLS], [SEP]
                                      max_length = max_length_text, # max length of the text that can go to BERT
                                      padding='max_length', # add [PAD] tokens
                                      return_attention_mask = True, # add attention mask to not focus on pad token
                                      )
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def convert_to_tfdataset(users_posts,labels):
  datasets = []
  for user_posts,label in zip(users_posts,labels):
    datasets.append( (encode_examples(user_posts,label)) )
  
  # dataset = tf.stack(datasets)
  return datasets



In [ ]:
dataset = convert_to_tfdataset(posts,lables)
total_users = len(dataset)
test_users = int(0.3 * total_users)
train_users = total_users - test_users
train_dataset = dataset[:train_users]
test_dataset = dataset[train_users:]

In [ ]:
# # Dataset
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# max_length_test = 50
# def map_example_to_dict_prediction(input_ids, attention_masks, token_type_ids):
#   return {
#       "input_ids": input_ids,
#       "token_type_ids": token_type_ids,
#       "attention_mask": attention_masks,
#   }

# def encode_examples_prediction(posts):
#   # prepare list, so that we can build up final TensorFlow dataset from slices.
#   input_ids_list = []
#   token_type_ids_list = []
#   attention_mask_list = []
#   for post in posts:
#     if not isinstance(post, str):
#       continue
    
#     bert_input = tokenizer.encode_plus(post,
#                                        truncation=True,           
#                                       add_special_tokens = True, # add [CLS], [SEP]
#                                       max_length = max_length_test, # max length of the text that can go to BERT
#                                       padding='max_length', # add [PAD] tokens
#                                       return_attention_mask = True, # add attention mask to not focus on pad token
#                                       )
#     input_ids_list.append(bert_input['input_ids'])
#     token_type_ids_list.append(bert_input['token_type_ids'])
#     attention_mask_list.append(bert_input['attention_mask'])
#   return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list)).map(map_example_to_dict_prediction)

# def convert_to_tfdataset(posts,labels,batch_size = 30):
#   dataset = encode_examples(posts,labels)
#   dataset = dataset.shuffle(dataset.__len__().numpy()).batch(batch_size)
#   return dataset

# # split
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split( posts, lables, test_size=0.2, random_state=21)
# train_dataset = convert_to_tfdataset(X_train,y_train,batch_size = 30)
# test_dataset = convert_to_tfdataset(X_test,y_test,batch_size = 30)

# # Train bert normally
# drive_models = 'drive/MyDrive/ire_bert/'
# # bert_history = bert_model.fit(train_dataset,epochs=10,verbose=1,validation_data=test_dataset)
# # bert_model.save_weights(drive_models+'bert_zenodo')

In [ ]:
class CRF_Model(tf.keras.Model):
  def __init__(self,N_LABELS):
    super(CRF_Model, self).__init__()
    self.emission_layer = bert_model
    initializer = tf.keras.initializers.GlorotUniform()
    self.transition_params = tf.keras.backend.variable( initializer([N_LABELS, N_LABELS]),name='transition_params')

  def call(self,input):
    # mask = tf.sequence_mask(inputs[1], tf.shape(inputs[0])[1])
    input = list(input)
    emissions = []
    for i in range(len(input)):
      for item in input[i]:
        input[i][item] = tf.expand_dims(input[i][item],axis = 0)
      emissions.append(self.emission_layer(input[i]))
    emissions = tf.squeeze(emissions)
    return [emissions,self.transition_params]

class Trainer(tf.keras.Model):
  def __init__(self,modelSpecs):
    super(Trainer, self).__init__()
    self.modelSpecs = modelSpecs
    self.model = CRF_Model(self.modelSpecs['N_LABELS'])

  def decode(self,emission, seq_lengths):
    viterbi_sequence_list = []
    viterbi_score_list = []
    for i in range(len(seq_lengths)):
      viterbi_sequence, viterbi_score = tfa.text.viterbi_decode(emission[i][:seq_lengths[i]], self.transition_params)
      viterbi_sequence_list.append(viterbi_sequence)
      viterbi_score_list.append(viterbi_score)
    return  viterbi_sequence_list, viterbi_score_list
  
  def predict(self, posts_seq):
    emission_scores = []
    for post in posts_seq:
      emissions = bert_model(post)
      emission_scores.append(emissions[0])
    emission_scores = tf.squeeze(emission_scores)
    viterbi_sequence, viterbi_score = tfa.text.viterbi_decode(emission_scores, self.model.transition_params)
    return viterbi_sequence

  def train(self,dataset):
    optimizer = tf.keras.optimizers.SGD(learning_rate=self.modelSpecs['learning_rate'])
    num_epochs = self.modelSpecs['n_epochs']
    logdir="logs"
    train_writer = tf.summary.create_file_writer(logdir+'/train')
    test_writer = tf.summary.create_file_writer(logdir+'/test')
    size = len(dataset)
    for epoch in range(num_epochs):
      for sample_no,inputs in enumerate(dataset):
        labels,tokens = [],[]
        for x in inputs.as_numpy_iterator():
          labels.append(x[1])
          tokens.append(x[0])
        # print(inputs, labels, seqLength)
        with tf.GradientTape() as tape:
          emission,transition_params = self.model(tokens)
          loss_value = self.lossfn(emission,labels,transition_params)
        tvars = self.trainable_variables
        tvars = [v for v in tvars if 'bert' not in v.name]
        print(tvars)
        grads = tape.gradient(loss_value, tvars)
        optimizer.apply_gradients(zip(grads, tvars))
        # predictedLabels, score = self.decode(emission, seqLength)
        with train_writer.as_default():
          tf.summary.scalar('losses', loss_value, step=sample_no+size*epoch)
  def call(self,inputs):
    # inputs: [emission, transition_params, seq_lengths]
    return self.decode(inputs[0],inputs[1],inputs[2])

  def lossfn(self, emission, labels, transition_params):
    cur_len_labels = len(labels)
    cur_len_emissions = len(emission)
    MAX_SEQ_LEN = 10
    if cur_len_labels < MAX_SEQ_LEN:
        paddings_label = (0, MAX_SEQ_LEN - cur_len_labels)
        labels = np.pad(labels, paddings_label)
    if cur_len_emissions < MAX_SEQ_LEN:
        paddings_emission = (( 0, MAX_SEQ_LEN - cur_len_emissions ),(0,0))
        emission = np.pad(emission, paddings_emission)
    labels = labels[:MAX_SEQ_LEN]
    emission = emission[:MAX_SEQ_LEN]
    emission = tf.expand_dims(emission, axis = 0)
    labels = np.array(labels, dtype = np.int)
    labels = tf.expand_dims(labels, axis = 0)
    log_likelihood, transition_params = tfa.text.crf_log_likelihood(emission, labels, [cur_len_labels], transition_params)
    loss = tf.reduce_mean(-log_likelihood)
    return loss


modelSpecs = {
    "learning_rate": 1e-4,
    "n_epochs": 2,
    "N_LABELS":2
}
import random
random.shuffle(train_dataset)
trainer = Trainer(modelSpecs)
trainer.train(train_dataset)

In [ ]:
posts = ["I am depressed","I wanna die"]
posts = [tokenizer(post, return_tensors="tf") for post in posts]
print(trainer.predict(posts))

[0, 0]


In [ ]:
modelSpecs = {
    "learning_rate": 1e-2,
    "n_epochs": 1,
    "N_LABELS":2
}

for x in dataset:
  y = x[0]
  y = list(y)
  for i in range(len(y)):
    for item in y[i]:
      y[i][item] = tf.expand_dims(y[i][item],axis = 0)
    bert_model(y[i])
  break

r = tokenizer("helo this is me", return_tensors="tf")
print(r)
# bert_model(r)

{'input_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[ 101, 2002, 4135, 2023, 2003, 2033,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [ ]:
# df_1 = pd.read_csv('/content/drive/My Drive/Mental Health blog dataset/beyondblue_depression.csv')
# df_2 = pd.read_csv( "/content/drive/My Drive/Mental Health blog dataset/beyondblue_ptsd-trauma.csv")
# df_3 = pd.read_csv('/content/drive/My Drive/Mental Health blog dataset/beyondblue_anxiety.csv')
# df_4 = pd.read_csv ('/content/drive/My Drive/Mental Health blog dataset/beyondblue_suicidal-thoughts-and-self-harm.csv')

# df_List = [df_1 , df_2, df_3, df_4]
# for data_frame in df_List : 
#   data_frame['index_indi'] = pd.Series ([i for i in range (data_frame.shape[0])])

# df = pd.concat(df_List, ignore_index=True)
# for i in range (df.date.shape[0]) : 
#   df.date[i] = datetime.strptime(df.date[i], '%d %B %Y')
# df.sort_values(by=['date'], inplace=True)
# unique_name = df.author.unique().tolist()
# unique_name = np.unique(np.array(unique_name))
# post_per_person = []
# for k, name in enumerate (unique_name) :
#   post_counts = df.loc[df['author'] == name].shape[0]
#   post_per_person.append ( [name, int (post_counts)] )

# temp = []
# for i in range (len(post_per_person)) : 
#   temp.append (post_per_person[i][1])
# sum (temp) / len(post_per_person)
# sub_data_names = [] 
# for i, freq in enumerate (temp) : 
#   if freq >= 5 and freq <= 50 : 
#     sub_data_names.append (post_per_person[i][0])

# sub_data_set = [] 
# for name in sub_data_names : 
#   sub_data_set.append (df.loc[df.author == name])

# train_cases = []
# for i in range(len ( sub_data_set) ) : 
#     cat = sub_data_set[i]['category'].to_numpy()
#     if 'suicidal-thoughts-and-self-harm' in cat and 'depression' in  cat: 
#       temp_1 = sub_data_set[i][sub_data_set[i].category == 'suicidal-thoughts-and-self-harm']
#       temp_2 = sub_data_set[i][sub_data_set[i].category == 'depression']
#       sub_data_set[i] = pd.concat([temp_1, temp_2])
#       sub_data_set[i].sort_values(by=['date'], inplace=True)
#       train_cases.append (sub_data_set[i])

# print('Num of users = ',len(train_cases))
# users = [] 
# labels = [] 
# for i in range (len(train_cases)) : 
#   users.append (train_cases[i].post.to_numpy())
#   labels.append (train_cases[i].category.to_numpy())

# category_labels = []
# sequence_length = []
# for i in range(len(labels)):
#   sequence_length.append(len(labels[i]))
#   cat = []
#   for label in labels[i]:
#     if label == 'suicidal-thoughts-and-self-harm':
#       cat.append(0)
#     else:
#       cat.append(1)
#   category_labels.append(cat)

In [ ]:
durl = 'drive/My Drive/zenodo_dataset/'
df_1 = pd.read_csv(durl + 'depression.csv')
df_2 = pd.read_csv(durl + 'suicidewatch.csv')

df_List = [df_1 , df_2]  
for data_frame in df_List : 
  data_frame['index_indi'] = pd.Series ([i for i in range (data_frame.shape[0])])

df = pd.concat(df_List, ignore_index=True)

In [ ]:
for i in range (df.date.shape[0]) : 
  df.date[i] = datetime.strptime(df.date[i], '%Y-%m-%d %H:%M:%S')
df.sort_values(by=['date'], inplace=True)
unique_name = df.author.unique().tolist()
unique_name = np.unique(np.array(unique_name))
post_per_person = []
for k, name in enumerate (unique_name) :
  post_counts = df.loc[df['author'] == name].shape[0]
  post_per_person.append ( [name, int (post_counts)] )
  
temp = []
for i in range (len(post_per_person)) : 
  temp.append (post_per_person[i][1])
sum (temp) / len(post_per_person)
sub_data_names = [] 
for i, freq in enumerate (temp) : 
  if freq >= 5 and freq <= 50 : 
    sub_data_names.append (post_per_person[i][0])

sub_data_set = [] 
for name in sub_data_names : 
  sub_data_set.append (df.loc[df.author == name])

train_cases = []
for i in range(len ( sub_data_set) ) : 
    cat = sub_data_set[i]['subreddit'].to_numpy()
    if 'SuicideWatch' in cat and 'depression' in  cat: 
      temp_1 = sub_data_set[i][sub_data_set[i].subreddit == 'SuicideWatch']
      temp_2 = sub_data_set[i][sub_data_set[i].subreddit == 'depression']
      sub_data_set[i] = pd.concat([temp_1, temp_2])
      sub_data_set[i].sort_values(by=['date'], inplace=True)
      train_cases.append (sub_data_set[i])

posts = [] 
lables = [] 
for i in range (len(train_cases)) : 
  posts.append (train_cases[i].post.to_numpy())
  lables.append (train_cases[i].subreddit.to_numpy())

print(len( post_per_person))

73556


In [ ]:
for label_seq in lables:
  for i in range(len(label_seq)):
    if label_seq[i] == 'depression':
      label_seq[i] = 0
    else:
      label_seq[i] = 1

In [ ]:
bert_labels = []
bert_posts = []
for post,label in zip(posts,lables):
  bert_labels.extend(label)
  bert_posts.extend(post)
print(len(bert_posts),len(bert_labels))

14259 14259


In [ ]:
x = 8
print(bert_posts[x],bert_labels[x])

It took me a few months to realize this but my pharmacist has been treating me very nice every time I visit to pickup my anti-depressant medication. She would always ask me how I’m doing and always smile at me. It’s not much but you don’t notice that with any other pharmacist. At first I found it odd that she was specifically being really nice to me but now I realize she was probably doing it because she knows I have depression and she wanted to cheer me up. So, to that pharmacist, I wanna thank you for that. You were the only person that interacts with me and it slightly lifts my mood when you treat me like a person. :) 0
